In [ ]:
!pip install git+https://github.com/quantumlib/OpenFermion-FQE --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import copy
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
from scipy import sparse
from scipy.linalg import expm

import openfermion as of
import fqe

# Fermi-Hubbard Model

The Hamiltonian of the system is given by:

$$H = - J \sum_{j = 1}^{L - 1} \sum_{\sigma \in \{ \uparrow, \downarrow \} } c_{j, \sigma}^\dagger c_{j + 1, \sigma} + \text{h.c.} + U \sum_{j} n_{j\uparrow} n_{j\downarrow}$$

where,

- $c_{j} (c_{j}^{\dagger})$ is the annihilation (creation) operator.
- $J$ hopping coefficient -  describes particles tunneling between neighboring sites.
- $U$ is the onsite interaction energy - the nature of the energy can be repulsive or attractive.
- $J = 1,2⋯,L$ is the number of orbitals/sites.
- $\sigma =\{\uparrow, \downarrow\}$ indicates the case when spins are different. Due to Fermi exclusion principle, a site can be occupied by spins of opposite spins.

The tunneling term represents the kinetic energy of the lattice. Due to tunneling term, when spin in state $\sigma$ is created at $j$-th site, another spin in the same state is annihilated at $j+1$-th site. It allows the spin conservation.


In [ ]:
# Defining Fermi-Hubbard Model in OpenFermion

nsites = 4
U = 0.01
J = 1.0

hubbard = of.hamiltonians.fermi_hubbard(1,nsites,tunneling=-J,coulomb=U, periodic= False)

In [ ]:
hubbard.terms

{((0, 1), (2, 0)): 1.0,
 ((2, 1), (0, 0)): 1.0,
 ((1, 1), (3, 0)): 1.0,
 ((3, 1), (1, 0)): 1.0,
 ((0, 1), (0, 0), (1, 1), (1, 0)): 0.01,
 ((2, 1), (4, 0)): 1.0,
 ((4, 1), (2, 0)): 1.0,
 ((3, 1), (5, 0)): 1.0,
 ((5, 1), (3, 0)): 1.0,
 ((2, 1), (2, 0), (3, 1), (3, 0)): 0.01,
 ((4, 1), (6, 0)): 1.0,
 ((6, 1), (4, 0)): 1.0,
 ((5, 1), (7, 0)): 1.0,
 ((7, 1), (5, 0)): 1.0,
 ((4, 1), (4, 0), (5, 1), (5, 0)): 0.01,
 ((6, 1), (6, 0), (7, 1), (7, 0)): 0.01}

- The spatial sites are mapped to fermionic modes (like spin-orbit splitting). They are connected by:
  $$N = 2L$$ where N is the number of fermionic modes and L is the number of spatial sites. i.e, each site is composed of one spin up fermionic mode and one spin down fermionic mode.

  > **Note:** Here we implicitly assumed the spin of fermions to be $\frac{1}{2}$, thus each sites will have a spin up and spin down modes. In essence the, number of sites and modes are related by number of spin states possible.

- In the language of quantum simulations, to simulate a L site Hubbard model we require 2L qubits where each qubit correspond to a mode.

- In the above example,
  - modes \{0,1\} represents site 1
  - modes \{2,3\} represents site 2
  - modes \{4,5\} represents site 3
  - modes \{6,7\} represents site 4

- In the above hamiltonian terms:
  - Term ((0, 1), (2, 0)) implies  a fermion is created in the site 1 and a fermion is annihilated at site 2. It essentially means that fermion hopped from site 2 to site 1. In the model considered, hopping occurs from $j+1$ to $j$, right to left.

  - Term ((0, 1), (0, 0), (1, 1), (1, 0)) represents the onsite interaction energy of spins occupied in the site 1.
  (0, 1), (0, 0) corresponds to number operator of up-spin and (1, 1), (1, 0) corresponds to number operator of down-spin.

## Time Evolution using FQE

In this section we are going to do time evolution with **Fermionic Quantum Emulator (FQE)**, a specialized simulator by openfermion to simulate fermionic quantum systems.

Let's assume the initial state to be a random state with two particle $S_z=0$ sector (total spin = 0, singlet state).

In [ ]:
nele, sz = 2, 0
init_wfn = fqe.Wavefunction([[nele,sz,nsites]])
init_wfn.set_wfn(strategy="random")

In [ ]:
init_wfn.print_wfn()

Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.3015781982539815-0.17326724282843906j)
a'0001'b'0010' (-0.0748360951307459-0.013106561970977235j)
a'0001'b'0100' (0.05868287509753664-0.0005949948620982391j)
a'0001'b'1000' (0.3214078664007357-0.2752648451275541j)
a'0010'b'0001' (0.2271268043042858-0.07276482282322508j)
a'0010'b'0010' (-0.10178952091147707+0.06697186441456113j)
a'0010'b'0100' (0.24570997905045913+0.0763587161726591j)
a'0010'b'1000' (0.051990081440134325+0.1209427799631717j)
a'0100'b'0001' (0.0250973113868779-0.015553236479918055j)
a'0100'b'0010' (0.06299062052315971-0.28204715090069205j)
a'0100'b'0100' (-0.29345352386681833-0.15547362999718944j)
a'0100'b'1000' (0.15953143735189595-0.05054711438059607j)
a'1000'b'0001' (-0.13887194770893158-0.2551166450750501j)
a'1000'b'0010' (0.026056459471780407-0.05746182410348805j)
a'1000'b'0100' (-0.009089811955446017+0.26980698076236215j)
a'1000'b'1000' (-0.17199498537854588+0.34926107844530146j)


We prepared a random initial state of a wavefunction that spans 4 sites.

- a'0001': This indicates that the spin-up electron (↑) is occupying the fourth site
- b'0001': Similarly, this indicates that the spin-down electron (↓) is also occupying the fourth site.

- The complex number represents amplitude such a configuration.



In [ ]:
# time evolution

%%time
e_time = 0.9
true_evolved_fqe = init_wfn.time_evolve(e_time,hubbard)

CPU times: user 92.8 ms, sys: 344 µs, total: 93.1 ms
Wall time: 107 ms



  FQE converts Hamiltonian into dense Hamiltonian representation to perform time-evolution. The evolution approximately is accomplished by Taylor expansion method.

In [ ]:
# Comparing fqe time evolution with speed of direct matrix exponentiation:

%%time

init_cirq_wfn = fqe.to_cirq(init_wfn) # convert into vector form
unitary = expm(-1j*e_time*of.get_sparse_operator(hubbard).todense())
true_evolved_cirq = unitary @ init_cirq_wfn


CPU times: user 302 ms, sys: 140 ms, total: 442 ms
Wall time: 266 ms


In [ ]:
true_evolved_fqe.expectationValue(fqe.get_hamiltonian_from_openfermion(hubbard))

(0.4562431772550703-2.7755575615628914e-17j)

In [ ]:
true_final_state = fqe.from_cirq(true_evolved_cirq, thresh=1e-12)
true_final_state.print_wfn()

Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.13288446280265964+0.20522236914789782j)
a'0001'b'0010' (-0.07297465695261485+0.23537324166549928j)
a'0001'b'0100' (0.03292976015167248-0.0027618779248792247j)
a'0001'b'1000' (0.21757146086763024-0.37324560657056016j)
a'0010'b'0001' (-0.19895909437356385+0.1965463984206332j)
a'0010'b'0010' (-0.019370507455977803+0.16912521153200447j)
a'0010'b'0100' (-0.18350107240741464-0.04112306991108876j)
a'0010'b'1000' (-0.0474656451696291-0.007334194756020993j)
a'0100'b'0001' (0.09232024629934145-0.026670765264990355j)
a'0100'b'0010' (0.24920080531747307-0.15084920461355197j)
a'0100'b'0100' (-0.14507744596980493+0.04738174721320555j)
a'0100'b'1000' (-0.0674044684956598-0.4349855399143071j)
a'1000'b'0001' (-0.23004332738651637-0.12915907984784294j)
a'1000'b'0010' (-0.11538646598386686-0.09134279224440274j)
a'1000'b'0100' (-0.11142857877222659-0.22483728674072825j)
a'1000'b'1000' (-0.23476233654083092-0.16252241300129197j)


In [ ]:
# checking the fidelity between final states of fqe evolution and exact diagonalization

fidelity = abs(fqe.vdot(true_evolved_fqe,true_final_state))**2
assert np.isclose(fidelity, 1.0)

### Conclusion

- Even for small sites, FQE evolution is faster than direct matrix exponentiation.

- Fidelity between final states in two approaches are closer to one.

# Trotterization for time evolution

Trotterization approach involves the Hubbard Hamiltonian into Hopping Hamiltonian `one_body_terms` and Onsite Interaction (charge-charge interaction) Hamiltonian `two_body_terms`.

## Splitting of Hubbard Hamiltonian

In [ ]:
# construction of fermi-hubbard hamiltonian
nqubits = 2*nsites # each site composed of two spins

# hopping terms of the Hamiltonian
one_body_terms = [op+of.hermitian_conjugated(op)
                for op in (of.FermionOperator(((i,1),(i+2,0)), coefficient=J)
                for i in range(nqubits-2))]

# onsite interaction term of the Hamiltonian
two_body_terms = [of.FermionOperator(((i,1),(i,0),(i+1,1),(i+1,0)),coefficient=U)
                 for i in range(0,nqubits,2)]

# verfiy this produces the same Hamiltonian from OpenFermion
assert sum(one_body_terms)+sum(two_body_terms) == hubbard

In [ ]:
sum(one_body_terms)+sum(two_body_terms)

2.0 [0^ 0 1^ 1] +
-1.0 [0^ 2] +
-1.0 [1^ 3] +
-1.0 [2^ 0] +
2.0 [2^ 2 3^ 3] +
-1.0 [2^ 4] +
-1.0 [3^ 1] +
-1.0 [3^ 5] +
-1.0 [4^ 2] +
2.0 [4^ 4 5^ 5] +
-1.0 [4^ 6] +
-1.0 [5^ 3] +
-1.0 [5^ 7] +
-1.0 [6^ 4] +
2.0 [6^ 6 7^ 7] +
-1.0 [7^ 5]

## Evolution of the Hopping Hamiltonian

### Direct approach

In [ ]:
sparse_hopping_matrix = of.get_sparse_operator(sum(one_body_terms))
unitary = expm(-1j*e_time*sparse_hopping_matrix.todense())
evolved_cirq_wfn = unitary @ init_cirq_wfn

### Givens Rotation to evolve Hopping Term

FQE can uses approximate methods to evolve the Hopping Hamiltonian. First of let's look into the matrix form of the Hopping Hamiltonian:

In [ ]:
hopping_matrix = J*(np.diag([1]*(nsites-1),k=1) + np.diag([1]*(nsites-1),k=-1))
print(hopping_matrix)

[[-0. -1. -0. -0.]
 [-1. -0. -1. -0.]
 [-0. -1. -0. -1.]
 [-0. -0. -1. -0.]]


- `np.diag([1]*(nsites-1),k=1)` superdiagonal elements.
- `np.diag([1]*(nsites-1),k=-1)` subdiagonal elements.

The matrix is symmetric, sparse and have only real entries. These matrices are ideal to be decomposed using Givens rotation.

**Givens Rotation** are used to decompose the Hamiltonian into a product of two-dimensional rotations, each acting on a pair of qubits or orbitals. This decomposition allows you to apply the time evolution operator in a piecewise fashion, evolving the wavefunction incrementally.

Using Givens Rotation helps to find the evolved state without direct exponentiation. FQE have function called `evolve_fqe_givens` which the approximate evolution of initial state through **Givens Rotation**.

In [ ]:
from fqe.algorithm.low_rank import evolve_fqe_givens

umat = expm(-1j*e_time*hopping_matrix)
evolved_wfn = evolve_fqe_givens(init_wfn,umat)

In [ ]:
# checking the fidelity

fidelity = abs(fqe.vdot(evolved_wfn, fqe.from_cirq(evolved_cirq_wfn,thresh=1e-6)))**2
assert np.isclose(fidelity,1.0)

## Evolution of the Onsite Interaction Term

### Direct Approach

In [ ]:
sparse_interaction_matrix = of.get_sparse_operator(sum(two_body_terms))
unitary = expm(-1j*e_time*sparse_interaction_matrix.todense())
evolved_wfn = unitary @ init_cirq_wfn
evolved_cirq_wfn = fqe.from_cirq(evolved_wfn, thresh=1e-6)

### Evolution using approximate methods in FQE

We use this to perform time-evolution with the `evolve_fqe_charge_charge_alpha_beta` function. This function applies


$$\exp\left(-i t \sum_{i,j} v_{i, j} n_{i, \alpha} n_{j, \beta} \right)$$

for any coefficients $v_{i,j}$ to the wavefunction.

Representing the exponential in this form allow to perform low rank approximation of matrix (for eg: trotter decomposition), which allow efficient calculations. Further, we only need to store $v_{j,j}$ values which are not equal to zero.


In [ ]:
interaction_matrix = np.diag([U]*nsites)
print(interaction_matrix) # diagonal matrix

[[2. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 2.]]


In [ ]:
from fqe.algorithm.low_rank import evolve_fqe_charge_charge_alpha_beta

evolved_fqe_wfn = evolve_fqe_charge_charge_alpha_beta(init_wfn,
                                                      interaction_matrix,
                                                      e_time)

In [ ]:
# Checking the fidelity

fidelity = abs(fqe.vdot(evolved_fqe_wfn,evolved_cirq_wfn))**2
print(fidelity)


1.0


## Trotterized Evolution of Hubbard Model

In [ ]:
trotter_steps = 10
umat = expm(-1j*(e_time/trotter_steps)*hopping_matrix)

current_wfn = copy.deepcopy(init_wfn)
for _ in range(trotter_steps):
  current_wfn = evolve_fqe_givens(current_wfn,  u=umat)
  current_wfn = evolve_fqe_charge_charge_alpha_beta(
        current_wfn,
        interaction_matrix,
        e_time / trotter_steps,
    )


trotterized_fidelity = abs(fqe.vdot(true_evolved_fqe, current_wfn))**2
print("Trotterized fidelity:", trotterized_fidelity)


Trotterized fidelity: 0.998143092806837
